In [ ]:
from requests import HTTPError

import cobra
import pandas as pd
import plotly.express as px
import seaborn as sns

from Bio.KEGG import REST
from matplotlib import rcParams
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Set default figure size
rcParams["figure.figsize"] = 10, 10

### Start of part 2 of report 

In [ ]:
#The above code is slow so saving it as a 'pickle' file for easy downstream analysis 

In [ ]:
fluxes.to_pickle('fluxes_with_description_df.pkl')

In [ ]:
fluxes=pd.read_pickle('fluxes_with_description_df.pkl')

In [ ]:
fluxes =fluxes.set_index('Reaction')

In [ ]:
#Setting the mouseover information to be the reaction definition.

In [ ]:
fig = px.scatter(fluxes, x="wildtype", y="perturbed", color="state", hover_data=["definition", fluxes.index])
fig.show()

In [ ]:
#over half the enteries have no KEGG ID

In [ ]:
len(fluxes.loc[fluxes["definition"]== "no_kegg_id"])

In [ ]:
# As 2946 reactions have no KEGG id, we want to get a fuller descriptions from the ICardio file - by merging it with the fluxes 

In [ ]:
icardio_data = pd.read_excel("ncomms14250-s10, iHsa COBRA.xlsx", sheet_name=1,engine="openpyxl")
icardio_data

In [ ]:
#From Icardio date frame, copying the corresponding subsystem information into the fluxes dataframe. This is done using the assign_subsystem fucntion

In [ ]:
fluxes["subsystem"] = ""  # create a new column to hold reaction suybsystem

# Assign subsystem to each row in the dataframe
def assign_subsystem(row):
    """Assign subsystem to a row (reaction)"""
    match = icardio_data.loc[icardio_data["Abbreviation"] == row.name]
    if not len(match):
        return "no_subsystem"
    subsystem = match["Subsystem"].item()
    if subsystem != "nan":
        return subsystem
    return "no_subsystem"

fluxes["subsystem"] = fluxes.apply(assign_subsystem, axis=1)

In [ ]:
#Interative scatterplot below will show the subsystem information on the data points 

In [ ]:
fig = px.scatter(fluxes, x="wildtype", y="perturbed", color="state", hover_data=["definition", fluxes.index, "subsystem"])
fig.show()

In [ ]:
#This function shows the differences in fluxes between the wildtype and perturbed models. 
#The function'dif' below performs the subtraction and adds the result to a new column in the flux dataframe

In [ ]:
fluxes["difference"]=""
#Assign subsystem to each row in the dataframe
def diff(row):
    return(int(row["wildtype"])- int(row['perturbed']))

fluxes["difference"]= fluxes.apply(diff,axis=1)

In [ ]:
#Removing "Reaction" as a index

In [ ]:
fluxes= fluxes.reset_index(drop=False)
fluxes

In [ ]:
#This will prevent truncation on row data in the dataframe

In [ ]:
pd.options.display.max_rows = 4200

In [ ]:
#This code below will group all reactions by state and the subsystem they belong to and display a count 

In [ ]:
state_and_sub_grouping= pd.DataFrame({'Qty_cnt':fluxes.groupby(["state","subsystem"])["Reaction"].count()}).reset_index()

In [ ]:
state_and_sub_grouping

In [ ]:
#The function below will enable plotting of a bar graph  for each of the states and show the counts of the reactions in the subsystems 

In [ ]:
def plot(pstate,phdr,pdf):
    df= pdf.loc[pdf["state"]==pstate]
    
    s = pd.Series(list(df['Qty_cnt']), index= list(df['subsystem']))
    
    plt.figure(figsize=(10,5))
    #set descriptions:
    plt.title(phdr)
    plt.ylabel('Count of reactions in subsystem')
    plt.xlabel('Subsystems')
    
    #Set tick colours:
    ax= plt.gca()
    ax.tick_params(axis='x', colors='black')
    ax.tick_params(axis='y', colors='red')
    
    #Plot the data:
    c=['green']
    
    s.plot(kind='bar',color=c)
    
    plt.show()

In [ ]:
#Bar graph of reactions that have been deactiavted 

In [ ]:
plot('deactivated','Count of Reactions in DEACTIVATED state',state_and_sub_grouping)

In [ ]:
#Bar graph of reactions that have been activated

In [ ]:
plot('activated','Count of reactions in ACTIVATED state',state_and_sub_grouping)

In [ ]:
#Bar graph of reactions that have decreased 

In [ ]:
plot('decreased flux','Count of reactions in DECREASED state',state_and_sub_grouping)

In [ ]:
#Bar graph of reactions that have increased

In [ ]:
plot('increased flux','Count of reactions in INCREASED state',state_and_sub_grouping)

In [ ]:
plot('unchanged','Count of reactions in UNCHANGED state',state_and_sub_grouping)

In [ ]:
#As there are a large number of reactions with no KEGG ID, merging the flux dataframe with Icardio gives more richer information on the reactions

In [ ]:
flux_merged_with_icardio= fluxes.merge (icardio_data, left_index=True, right_on= "Abbreviation")
flux_merged_with_icardio

In [ ]:
#set abbreviation as index on the new merged dataframe 

In [ ]:
flux_merged_with_icardio= flux_merged_with_icardio.set_index('Abbreviation')
flux_merged_with_icardio

In [ ]:
#Showing all 4200 reactions along with their corresponding subsystems and state

In [ ]:
all_state=pd.DataFrame({'Difference=wildtype- perturbed' : flux_merged_with_icardio.groupby(["state","subsystem","Description"])['difference'].sum()})
all_state

In [ ]:
#Converting table above into pickle file so it can be opened in excel for further understanding

In [ ]:
all_state.to_csv('allstate_with_Icardio_description_df.csv')

In [ ]:
#Printing all transport reaction descriptions that have increased 

In [ ]:
inc_transport = flux_merged_with_icardio.loc[(flux_merged_with_icardio["subsystem"] == "Transport") & (flux_merged_with_icardio["state"] == "increased flux")]
inc_transport = pd.DataFrame(inc_transport.groupby( ["state", "subsystem", "Description", "wildtype", "perturbed"])['difference'].sum())
inc_transport

We can see from the results that transport reactions have the most increased flux from shutting off glucose import followed by central carbon metabolism.
The heart is able to switch energy sources flexibly as dictated by substrate availability therefore the low availability of glucose results in the increase import of fatty acids  into cardio myocytes as fuel molecules. This is critical when blood glucose is low.

L-carnitine transport is also increased once glucose has been shut off. This is a chemical that is essential for the transfer of fatty acids across he mitochondria where they can be oxidized to produce energy.

In [ ]:
#Printing all protein metabolism reaction descriptions that have decreased 

In [ ]:
dec_protein = flux_merged_with_icardio.loc[(flux_merged_with_icardio["subsystem"] == "Protein metabolism") & (flux_merged_with_icardio["state"] == "decreased flux")]
dec_protein = pd.DataFrame(dec_protein.groupby( ["state", "subsystem", "Description", "wildtype", "perturbed"])['difference'].sum())
dec_protein

From these results we can see that amino acid ligases have decreases metabolism. These ligases catalyse the attachment of an amino acid to tRNA in order to make protein, this is an ATP- dependant process.

In a postabsorptive state, insulin is secreted and this stimulates the synthesis of many proteins in a variety of ways for example it stimulates glycogen synthesis and accelerates glycolysis in the liver which then increases fatty acid synthesis. Insulin also increases protein and amino acid metabolism. 

However in this case where there is no glucose available, the cardiomyocytes may be preserving energy and wont synthesise proteins. Under these extreme conditions when glucose levels are low such as in Kwashiorkor which is a severe form on malnutrition that tends to occur in developing countries, cells want to switch off all biosynthetic pathways in order to prevent wasting energy making proteins and instead use the amino acids to make energy. (Berg et al,2002)

In [ ]:
#Printing all transport reaction descriptions that have been deactivated 

In [ ]:
deactivated_transport = flux_merged_with_icardio.loc[(flux_merged_with_icardio["subsystem"] == "Transport") & (flux_merged_with_icardio["state"] == "deactivated")]
deactivated_transport = pd.DataFrame(deactivated_transport.groupby( ["state", "subsystem", "Description", "wildtype", "perturbed"])['difference'].sum())
deactivated_transport

Deactivating malate / Pi antiport and Pi / succinate antiport may be due to try and keep the carbons in the cells a source of energy 

In [ ]:
#Printing all transport reaction descriptions that have been activated 

In [ ]:
activated_transport = flux_merged_with_icardio.loc[(flux_merged_with_icardio["subsystem"] == "Transport") & (flux_merged_with_icardio["state"] == "activated")]
activated_transport = pd.DataFrame(activated_transport.groupby( ["state", "subsystem", "Description", "wildtype", "perturbed"])['difference'].sum())
activated_transport

Transport reactions again are being activated the most after setting off glucose import, many of these transport reactions are amino acid transport. Amino acids play a central role in cardiac metabolism, the heart just like for fatty acids can use amino acids for energy. More amino acids are transported into the cardiomyocytes in order to make up for the missing glucose.
Fumarate / malate antiport reaction may be trying to scavage as many of the intermiades in the TCA cycle from the cytosol as it can so that it can get that into the mitochondria for ATP

#### Constraint change 

Now changing constraints again, this time allowing a small smount of glucose to enter the cells by setting constraint to 10 and -10 and identifying any differences.

In [ ]:
#The graph was screenshotted from a seperate notebook and pasted onto the report word document.

## Conclusion

Computational models of biological systems such as ICardio have proven their worth in highlighting complications of conditions such as a Myocardial infarction. Myocardial infarction causes a sudden blockage of blood into the heart and results in the heart not getting the nutrients that it needs to function. When this occurs, and the heart loses its primary source of energy, glucose, it needs to find other substrates for energy. Using a mathematical model of the heart, ICardio, we have found there is an increase of transport of fatty acids, amino acids, and pyruvate into the cell to counteract the missing glucose. Also, the decrease and deactivation of reactions involving protein synthesis prevents any energy being wasted. Using models such as this to represent other heart abnormalities can allow for further understanding of these conditions and therefore better treatment.
